In [38]:
dpath='E:\soft\selenium\chromedriver.exe'

In [39]:
import time
import pymongo
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from lxml import etree
from selenium.webdriver.chrome.options import Options

In [40]:
browser = webdriver.Chrome(dpath)
wait = WebDriverWait(browser, 30)

In [34]:
item={'1':'111','2':'222'}

In [35]:
db.insert_one(item)

In [41]:
db = pymongo.MongoClient('mongodb://localhost:27017/')['mydb']['jd']

In [23]:
def search():
    browser.get('https://www.jd.com/')
    try:
        input = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#key"))
        )  # llist
        submit = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#search > div > div.form > button"))
        )
        # input = d.find_element_by_id('key')
        # submit = d.find_element_by_id('search-link')
        input[0].send_keys('python')
        submit.click()

        total = wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, '#J_bottomPage > span.p-skip > em:nth-child(1) > b')
            )
        )
        html = browser.page_source
        prase_html(html)
        return total[0].text
    except TimeoutError:
        search()

In [47]:
def next_page(page_number):
    try:
        # 滑动到底部，加载出后三十个货物信息
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        # 翻页动作
        button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#J_bottomPage > span.p-num > a.pn-next > em'))
        )
        button.click()
        wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#J_goodsList > ul > li:nth-child(60)"))
        )
        # 判断翻页成功
        wait.until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, "#J_bottomPage > span.p-num > a.curr"), str(page_number))
        )
        html = browser.page_source
        prase_html(html)
    except TimeoutError:
        return next_page(page_number)

In [30]:
def prase_html(html):
    html = etree.HTML(html)
    items = html.xpath('//li[@class="gl-item"]')
    for i in range(len(items)):
        item = {}
        if html.xpath('//div[@class="p-img"]//img')[i].get('data-lazy-img') != "done":
            img = html.xpath('//div[@class="p-img"]//img')[i].get('data-lazy-img')
        else:
            img = html.xpath('//div[@class="p-img"]//img')[i].get('src')

        # 图片url
        item["img"] = img
        # 标题
        item["title"] = html.xpath('//div[@class="p-name p-name-type-2"]//em')[i].xpath('string(.)')
        # 价格
        item["price"] = html.xpath('//div[@class="p-price"]//i')[i].text
        # 评论
        item["commit"] = html.xpath('//div[@class="p-commit"]//a')[i].text

        save(item)

In [36]:
def save(item):
    try:
        db.insert_one(item)
    except Exception:
        print('{}存储到MongoDB失败'.format(str(item)))

In [43]:
def main():
    print("第", 1, "页：")
    total = int(search())
    for i in range(2, total + 1):
        #time.sleep(3)
        print("第", i, "页：")
        next_page(i)

In [46]:
main()

第 1 页：
第 2 页：


TimeoutException: Message: 
